<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8e3877ee587b3d70d1b0c5d3f67ce6e3a8a6dda345c54bc66dba936a02d065b4
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-09 11:00:57
-------------------
qualified stocks: 90
with latest results: 27
still star stocks: 16


-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  5.27 L
Current:  1.40 C
-------------------
Today PnL: 22.95 K (0.16%)
Current PnL: -17.89 L (-12.04%)
CY Booked + Current PnL: -8.03 L (-5.4%)
-------------------
Total profit:  2.75 L
Total loss:  -20.64 L
-------------------
Total Booked + Current PnL: 18.95 L (15.57%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.04%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 79.18 L (56.53%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.81%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,DIXON,18940.15,-28.50,69.0,H-MC,8.48,143040.0,28104.0,8482.0,-0.70,24.45,5.93,31.83,81.0,3.31,1.06,44.23,X40N,ATH,IT
5,APOLLOHOSP,8285.00,-13.86,59.0,H-LC,10.78,172018.0,15841.0,10252.0,0.42,10.14,5.96,16.71,22.0,1.55,1.28,29.35,X40N,BTT,HEALTHCARE
80,TTKPRESTIG,770.00,106.19,64.0,M-SC,5.88,88870.0,-11907.0,11997.0,0.91,-11.82,13.50,0.09,245.0,-0.99,0.66,16.76,OX40N,NTT,DURABLES
65,SBIN,863.00,-13.25,48.0,M-LC,4.29,214200.0,12720.0,14501.0,-0.06,6.31,6.77,13.51,60.0,0.88,1.59,19.71,XY25,NTT,BANKS
34,HEROMOTOCO,5949.07,3.19,81.0,H-MC,22.94,190908.0,39995.0,17315.0,0.21,26.50,9.07,37.97,93.0,2.31,1.42,58.51,AR,ATH,AUTO


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,41.94,62.0,M-MC,8.03,224497.0,-465.0,168126.0,-0.82,-0.21,74.89,74.53,192.0,-0.00,1.67,31.98,XY24,BTT,STEEL
4,ANGELONE,3033.00,8.16,40.0,H-SC,6.49,191642.0,636.0,57071.0,-0.17,0.33,29.78,30.21,157.0,0.01,1.42,21.28,X40N,NTT,FINANCE
32,HAVELLS,2069.16,-0.94,58.0,H-MC,1.81,245513.0,-2218.0,77287.0,-0.34,-0.90,31.48,30.30,92.0,-0.03,1.82,12.28,X40,ATH,ELECTRICAL
37,ICICIGI,2260.25,-21.27,39.0,H-MC,3.71,135568.0,-565.0,31696.0,-0.05,-0.42,23.38,22.86,91.0,-0.02,1.01,12.15,X40,ATH,INSURANCE
54,PGHH,17973.08,-30.94,52.0,H-MC,4.80,200040.0,-780.0,69554.0,0.45,-0.39,34.77,34.25,80.0,-0.01,1.48,4.79,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HCLTECH,1943.91,-3.66,30.0,H-LC,10.09,220271.0,-21653.0,84914.0,-1.13,-8.95,38.55,26.15,8.0,-0.25,1.63,4.09,X40,ATH,IT
42,INFY,2275.00,-21.75,37.0,H-LC,10.89,305208.0,-7680.0,179371.0,-0.81,-2.45,58.77,54.87,3.0,-0.04,2.26,4.03,X40,BTT,IT
86,VBL,671.64,-15.83,39.0,H-LC,6.66,302890.0,-12952.0,125608.0,0.15,-4.10,41.47,35.67,5.0,-0.10,2.25,9.19,X40N,ATH,FMCG
46,JIOFIN,387.00,-1.80,41.0,H-LC,13.89,217280.0,-126.0,53625.0,-0.23,-0.06,24.68,24.61,48.0,-0.00,1.61,54.73,XY24,BTT,FINANCE
83,UNITDSPR,1644.00,-14.51,44.0,H-LC,8.34,171877.0,-9882.0,45135.0,0.47,-5.44,26.26,19.39,86.0,-0.22,1.28,2.27,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LTIM,7230.20,-6.94,47.0,H-LC,1.63,230715.0,-15436.0,94639.0,0.75,-6.27,41.02,32.18,16.0,-0.16,1.71,27.51,X200,ATH,IT
53,NESTLEIND,1377.00,-9.37,57.0,H-LC,2.76,278881.0,13455.0,43338.0,0.35,5.07,15.54,21.40,11.0,0.31,2.07,13.25,XY25,NTT,FMCG
44,ITC,452.00,-37.92,50.0,H-LC,3.23,200099.0,-39.0,20930.0,0.40,-0.02,10.46,10.44,4.0,-0.00,1.48,5.54,X40,NTT,FMCG
20,CIPLA,1795.00,-20.81,47.0,H-LC,5.78,211720.0,7220.0,34193.0,0.27,3.53,16.15,20.25,10.0,0.21,1.57,12.57,X40N,BTT,PHARMA
86,VBL,671.64,-15.83,39.0,H-LC,6.66,302890.0,-12952.0,125608.0,0.15,-4.10,41.47,35.67,5.0,-0.10,2.25,9.19,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UJJIVANSFB,60.0,101.87,68.0,H-SC,12.85,127278.0,-15201.0,34721.0,6.72,-10.67,27.28,13.70,163.0,-0.44,0.94,50.22,OX40N,NTT,BANKS
84,VAIBHAVGBL,521.0,57.02,66.0,H-SC,11.01,104794.0,-47197.0,121320.0,5.76,-31.05,115.77,48.77,125.0,-0.39,0.78,21.97,XR,NTT,JEWELLERY
85,VALIANTORG,838.0,-339.55,66.0,H-SC,8.92,101712.0,-31574.0,116165.0,4.99,-23.69,114.21,63.47,139.0,-0.27,0.75,70.49,XR,NTT,CHEMICALS
78,TITAGARH,1548.0,-6.26,58.0,H-SC,7.84,153598.0,-38417.0,117303.0,3.87,-20.01,76.37,41.08,158.0,-0.33,1.14,30.15,XY24,NTT,ENGINEERING
66,SFL,1287.0,44.58,64.0,M-SC,9.04,190868.0,-71350.0,141185.0,3.29,-27.21,73.97,26.63,229.0,-0.51,1.42,18.67,XY24,NTT,MISC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3754.05,65.0,M-SC,7.78,120136.0,-27232.0,79049.0,-4.89,-18.48,65.80,35.16,236.0,-0.34,0.89,22.71,XY24,NTT,AUTO
22,COFFEEDAY,80.00,-39.64,60.0,L-SC,40.41,90163.0,-23386.0,63439.0,-3.79,-20.60,70.36,35.27,268.0,-0.37,0.67,120.68,XR,NTT,HOTELS
67,SHALBY,327.00,1059.49,70.0,M-SC,2.84,157616.0,-23527.0,68989.0,-2.94,-12.99,43.77,25.10,235.0,-0.34,1.17,26.31,XY24,NTT,HEALTHCARE
71,STARHEALTH,761.00,8.28,51.0,H-SC,3.74,160954.0,-39262.0,115291.0,-1.82,-19.61,71.63,37.97,144.0,-0.34,1.19,29.23,XY24,NTT,INSURANCE
52,NATIONALUM,247.44,-44.49,67.0,H-MC,1.74,110166.0,9662.0,20976.0,-1.28,9.61,19.04,30.49,79.0,0.46,0.82,45.13,MH,ATH,METALS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UJJIVANSFB,60.0,101.87,68.0,H-SC,12.85,127278.0,-15201.0,34721.0,6.72,-10.67,27.28,13.70,163.0,-0.44,0.94,50.22,OX40N,NTT,BANKS
80,TTKPRESTIG,770.0,106.19,64.0,M-SC,5.88,88870.0,-11907.0,11997.0,0.91,-11.82,13.50,0.09,245.0,-0.99,0.66,16.76,OX40N,NTT,DURABLES
47,KANSAINER,340.0,-67.24,65.0,H-SC,2.56,225558.0,-44109.0,80434.0,3.00,-16.36,35.66,13.47,138.0,-0.55,1.67,14.08,XY24,NTT,PAINTS
40,INDIGOPNTS,1408.0,115.40,50.0,M-SC,3.23,141112.0,-33447.0,33486.0,-0.32,-19.16,23.73,0.02,221.0,-1.00,1.05,22.74,OX40N,NTT,PAINTS
19,CERA,9475.0,-19.81,47.0,H-SC,2.40,114534.0,-29269.0,56019.0,-0.34,-20.35,48.91,18.60,149.0,-0.52,0.85,26.39,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TCS,4998.00,-28.04,44.0,H-LC,14.18,287020.0,-58936.0,182803.0,1.14,-17.04,63.69,35.80,1.0,-0.32,2.13,1.68,X40,BTT,IT
42,INFY,2275.00,-21.75,37.0,H-LC,10.89,305208.0,-7680.0,179371.0,-0.81,-2.45,58.77,54.87,3.0,-0.04,2.26,4.03,X40,BTT,IT
44,ITC,452.00,-37.92,50.0,H-LC,3.23,200099.0,-39.0,20930.0,0.40,-0.02,10.46,10.44,4.0,-0.00,1.48,5.54,X40,NTT,FMCG
86,VBL,671.64,-15.83,39.0,H-LC,6.66,302890.0,-12952.0,125608.0,0.15,-4.10,41.47,35.67,5.0,-0.10,2.25,9.19,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,44.0,H-LC,12.77,245578.0,-16041.0,135264.0,0.15,-6.13,55.08,45.57,7.0,-0.12,1.82,4.50,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,3.19,81.0,H-MC,22.94,190908.0,39995.0,17315.0,0.21,26.50,9.07,37.97,93.0,2.31,1.42,58.51,AR,ATH,AUTO
26,DIXON,18940.15,-28.50,69.0,H-MC,8.48,143040.0,28104.0,8482.0,-0.70,24.45,5.93,31.83,81.0,3.31,1.06,44.23,X40N,ATH,IT
27,DMART,5201.00,-6.62,64.0,H-LC,16.58,233289.0,48808.0,21556.0,0.99,26.46,9.24,38.14,38.0,2.26,1.73,39.87,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,2326.38,9.17,53.0,H-MC,3.36,188450.0,23780.0,44192.0,0.02,14.44,23.45,41.28,89.0,0.54,1.40,23.58,X40N,ATH,AC
4,ANGELONE,3033.00,8.16,40.0,H-SC,6.49,191642.0,636.0,57071.0,-0.17,0.33,29.78,30.21,157.0,0.01,1.42,21.28,X40N,NTT,FINANCE
34,HEROMOTOCO,5949.07,3.19,81.0,H-MC,22.94,190908.0,39995.0,17315.0,0.21,26.50,9.07,37.97,93.0,2.31,1.42,58.51,AR,ATH,AUTO
87,VOLTAS,1918.49,1.44,61.0,H-MC,3.47,212745.0,21003.0,75035.0,-0.06,10.95,35.27,50.08,99.0,0.28,1.58,18.50,XY25,ATH,AC
24,DABUR,735.00,-2.76,60.0,H-MC,5.13,211516.0,14488.0,75130.0,-0.71,7.35,35.52,45.49,102.0,0.19,1.57,21.35,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,3.19,81.0,H-MC,22.94,190908.0,39995.0,17315.0,0.21,26.50,9.07,37.97,93.0,2.31,1.42,58.51,AR,ATH,AUTO
52,NATIONALUM,247.44,-44.49,67.0,H-MC,1.74,110166.0,9662.0,20976.0,-1.28,9.61,19.04,30.49,79.0,0.46,0.82,45.13,MH,ATH,METALS
88,WHIRLPOOL,2270.00,-39.15,63.0,M-SC,3.11,102750.0,11252.0,67496.0,0.51,12.30,65.69,86.07,223.0,0.17,0.76,49.21,XR,NTT,DURABLES
39,INDIAMART,4850.92,-51.09,50.0,H-SC,12.13,136687.0,13351.0,120408.0,0.17,10.83,88.09,108.45,119.0,0.11,1.01,35.60,AR,ATH,MISC
89,WIPRO,420.00,-14.25,39.0,M-LC,6.89,150362.0,-583.0,110035.0,-0.47,-0.39,73.18,72.51,53.0,-0.01,1.12,5.61,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6722.22,52.0,L-SC,6.20,74827.0,-18983.0,95599.0,1.30,-20.24,127.76,81.67,269.0,-0.20,0.56,44.90,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-5.85,41.0,H-SC,15.00,87673.0,-13290.0,107820.0,0.17,-13.16,122.98,93.63,148.0,-0.12,0.65,30.59,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,60.0,L-SC,40.41,90163.0,-23386.0,63439.0,-3.79,-20.60,70.36,35.27,268.0,-0.37,0.67,120.68,XR,NTT,HOTELS
51,MASFIN,398.61,-15.99,60.0,H-SC,12.95,95310.0,-2670.0,24275.0,2.37,-2.73,25.47,22.05,152.0,-0.11,0.71,38.54,XR,ATH,FINANCE
85,VALIANTORG,838.00,-339.55,66.0,H-SC,8.92,101712.0,-31574.0,116165.0,4.99,-23.69,114.21,63.47,139.0,-0.27,0.75,70.49,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,60.0,L-SC,40.41,90163.0,-23386.0,63439.0,-3.79,-20.60,70.36,35.27,268.0,-0.37,0.67,120.68,XR,NTT,HOTELS
8,ATULAUTO,844.00,3754.05,65.0,M-SC,7.78,120136.0,-27232.0,79049.0,-4.89,-18.48,65.80,35.16,236.0,-0.34,0.89,22.71,XY24,NTT,AUTO
34,HEROMOTOCO,5949.07,3.19,81.0,H-MC,22.94,190908.0,39995.0,17315.0,0.21,26.50,9.07,37.97,93.0,2.31,1.42,58.51,AR,ATH,AUTO
63,SAMMAANCAP,326.00,-171.59,64.0,M-SC,4.59,82104.0,-20106.0,113492.0,-0.64,-19.67,138.23,91.37,208.0,-0.18,0.61,33.28,XY25,NTT,FINANCE
85,VALIANTORG,838.00,-339.55,66.0,H-SC,8.92,101712.0,-31574.0,116165.0,4.99,-23.69,114.21,63.47,139.0,-0.27,0.75,70.49,XR,NTT,CHEMICALS


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.39
1,25,42.86
2,50,73.07


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.27
LC    35.41
MC    25.33
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.61
X40N     15.43
X40      15.22
XY25      9.90
AR        9.36
XR        9.14
OX40N     7.45
MH        1.72
X200      1.71
X5K       1.40
SR        1.07
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.22
H-SC    24.00
H-MC    22.34
M-SC    13.66
M-LC     7.12
M-MC     2.67
L-SC     1.61
L-LC     1.07
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.56,-0.85,35.46
IT,13.14,-16.46,74.20
FINANCE,8.73,-18.82,61.54
BANKS,7.36,-14.88,63.66
PAINTS,5.99,-14.43,40.04
MISC,5.80,-17.57,84.96
ELECTRICAL,5.60,-9.04,69.14
AUTO,4.82,-3.98,49.22
AC,3.70,2.79,30.14


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2743619.0,22
AR,1216257.0,10
XR,1063691.0,13
X40,912929.0,10
X40N,604172.0,12
OX40N,453728.0,10
XY25,418446.0,7
SR,198550.0,2
X5K,128500.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2786710.0,28
H-LC,1449850.0,16
M-SC,1404849.0,17
H-MC,1194567.0,17
M-LC,419430.0,5
M-MC,318118.0,2
L-SC,244543.0,3
L-MC,59854.0,1
L-LC,40507.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,922017.0,6
M-SC,XY24,819442.0,7
H-SC,AR,660743.0,5
H-LC,X40,618201.0,5
H-SC,XR,499499.0,6
H-MC,XY24,464023.0,4
H-LC,AR,378387.0,2
M-MC,XY24,318118.0,2
H-SC,X40N,249187.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 43.0 seconds
